# 单机版Tensorflow任务

您可以在Notebook中使用TI SDK来提交训练任务。TI SDK不仅支持Tensorflow，PyTorch等流行的机器学习框架，还允许用户使用自定义镜像。这个示例展示如何使用TI SDK来提交单机版的Tensorflow任务

#### 引入依赖

In [1]:
from __future__ import absolute_import, print_function

import sys
from ti import session
from ti.tensorflow import TensorFlow

#### 准备训练数据和脚本

我们已经在对象存储COS中准备好了一份训练数据(点击[此链接](https://ti-ap-guangzhou-1300268737.cos.ap-guangzhou.myqcloud.com/training_data/tensorflow/simple/heart.csv)下载)，您也可以替换为自己的数据。

提交一个任务的步骤包括：
1. 准备训练数据
2. 创建一个Tensorflow Estimator，这里可以指定各种需要的参数
3. 调用fit()提交训练任务

Tensorflow Estimator目前支持的参数有
- role： 用户授权给TI的角色名称
- train_instance_count：计算实例的个数
- train_instance_type： 计算实例类型
- py_version: python版本，默认'py3'
- script_mode: 是否是脚本模式，目前仅支持该模式，默认True
- hyperparameters: 算法超级参数，python字典类型，算法需要的所有参数在这里添加
- framework_version: tensorflow版本
- entry_point: 训练脚本文件名
- source_dir: 训练脚本的本地路径
- output_path: 训练完模型的存储路径 cos://$bucket/$key_prefix

TI SDK会拼接py_version和framework_version生成一个默认的镜像名，比如下面的例子中的镜像名为

_tensorflow:2.0.0b-py3_

您也可以通过**image_name**参数指定镜像名称，此时py_version和framework_version会被忽略:
- image_name: 比如'ccr.ccs.tencentyun.com/user/tensorflow:1.14.0-py3', 请确保在腾讯云镜像仓库中已经上传该镜像

In [3]:
import os
# 初始化一个session
ti_session = session.Session()

# 授权给TI的服务角色
role = "TIONE_QCSRole"

# 读取COS中存储的训练数据
inputs = 'cos://ti-%s-1300268737/training_data/tensorflow/simple' % (os.environ.get('REGION'))

# 设置算法超级参数
hyperparameters = {'batch_size': 64}

# 创建一个Tensorflow Estimator
tf_estimator = TensorFlow(role=role,
                          train_instance_count=1,
                          train_instance_type='TI.SMALL2.1core2g',
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters,
                          framework_version='2.0.0b',
                          entry_point='train.py',
                          source_dir='code')

# 提交Tensorflow训练任务
tf_estimator.fit(inputs)

## 查看输出模型


如果训练任务完成有模型输出，那么TI会将模型上传到cos上。
你可以通过**output_path='cos://'**指定模型存储的cos路径。如果没指定，TI会按以下格式创建存储通
**ti-[region]-[uin]-[appid]**

最终模型会放在 #{bucket}/#{job_name}/output/目录下


## 查看训练日志


TI训练过程中会把日志上传到[腾讯云日志服务](https://cloud.tencent.com/product/cls)中，使用腾讯云日志服务需要先开通。

TI会针对训练任务创建TiOne日志集和TrainingJob日志主题，通过日志检索功能可以搜索对应训练任务的日志。


目前TI会默认创建一个日志集(TI)和日志主题(TrainingJob)

TI内置了任务名称(job)关键词，可以通过以下条件过滤指定任务的日志

```
job: #{job_name}
```

更多日志检索语法请参考https://cloud.tencent.com/document/product/614/16981

## 查看提交历史和监控

您可以在TI产品控制台上的【任务列表】-【SDK任务】中查看您提交的SDK任务历史，点击【任务名称】列中的任务名可以看到任务的详细信息，在列表的右侧【监控】可以查看任务运行的资源监控图表